***Панихидин Дмитрий Борисович***

Тестовое задание Гринатом CaseLab ML

Классификация отзывов на фильмы

In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
import nltk
from nltk.stem import snowball
#nltk.download('udhr')
#nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from joblib import dump, load
from sklearn.svm import SVC

# Предобработка данных

In [27]:
os.getcwd()

'D:\\CaseLabChallenge\\Scripts'

In [28]:
train_pos = '../dataset/aclImdb/train/pos'
train_neg = '../dataset/aclImdb/train/neg'
train_pos, train_neg

('../dataset/aclImdb/train/pos', '../dataset/aclImdb/train/neg')

In [29]:
len(os.listdir(train_pos)), len(os.listdir(train_neg))

(12500, 12500)

In [30]:
path_example = train_pos+'/'+os.listdir(train_pos)[0]
path_example
with open(path_example) as f:
    text_example = f.read()

text_example

'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'

In [32]:
def load_reviews_from_files(path, label):
    file_paths = [path + '/' + fname for fname in os.listdir(path)]
    reviews = []
    for p in file_paths:
        with open(p, 'r', encoding='utf-8') as f:
            review = f.read()
            reviews.append((review,label))
    return reviews


In [ ]:
pos_reviews = load_reviews_from_files(train_neg, 0)
pos_reviews

In [ ]:
os.listdir(train_pos)

In [34]:
neg_reviews = load_reviews_from_files(train_pos, 1)

In [ ]:
neg_reviews

In [35]:
len(pos_reviews), len(neg_reviews)


(12500, 12500)

In [36]:
reviews_train = pos_reviews + neg_reviews 
len(reviews_train)

25000

In [37]:
df_train = pd.DataFrame(reviews_train, columns=['review', 'label'])

In [38]:
df_train.head(3)

,review,label
0,Story of a man who has unnatural feelings for ...,0
1,Airport '77 starts as a brand new luxury 747 p...,0
2,This film lacked something I couldn't put my f...,0


In [39]:
df_train.tail(3)

,review,label
24997,I am amazed at how this movie(and most others ...,1
24998,A Christmas Together actually came before my t...,1
24999,Working-class romantic drama from director Mar...,1


In [40]:
df_train = df_train.reindex(np.random.permutation(df_train.index))
df_train.reset_index(drop=True, inplace=True)

In [41]:
df_train.head(10)

,review,label
0,"First of all, this is a low-budget movie, so m...",0
1,This is a bad B movie masquerading as a mockum...,0
2,...but I've seen better too.<br /><br />The st...,0
3,Channel surfing and caught this on LOGO. It wa...,0
4,"""The Invisible Ray"" is part science fiction an...",1
5,I only rented this movie because of promises o...,0
6,This is a direct sequel to 'The Mummy's Hand' ...,0
7,The first collaboration between Schoedsack & C...,1
8,There wasn't a day in 2002 where i wasn't chas...,1
9,[***POSSIBLE SPOILERS***] This movie's reputat...,0


In [42]:
df_train.tail(10)

,review,label
24990,A charming little film set in the UK about the...,1
24991,So what's the big fuss out of making an INDIAN...,0
24992,My title above says it all. Let me make it cle...,0
24993,"Damn, I thought I'd seen some bad westerns. Ca...",0
24994,Paris Je T ' aime is a movie that explores the...,1
24995,"I can sit through this movie once, but I doubt...",0
24996,I thought How The Grinch Stole Christmas was a...,1
24997,The film is a bit tedious. It's mostly a silen...,0
24998,"The original ""les visiteurs"" was original, hil...",0
24999,"Criminals Perry Smith and Richard ""Dick"" Hicko...",1


In [43]:
import re
# Удалим стоп слова
stop_words = set(stopwords.words('english'))

#Используем стеммер
stemmer = snowball.SnowballStemmer('english')

# функция для предобработки
def preprocess(text):
    text = re.sub(r'<.*?>', '', text)
    emoticons = re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text = (re.sub(r'[\W]+', ' ', text)) +' ' + ' '.join(emoticons).replace('-','')
    tokens = [word for word in text.split() if word.lower() not in stop_words]
    tokens = [stemmer.stem(token) for token in tokens]
    text = ' '.join(tokens)
    return text

In [44]:
test_text = "This;;; :( is <HTML> :( <br> :)</br> <qqq> :-)  a test runners doggers cookies likes"
preprocess(test_text)

'test runner dogger cooki like :( :( :) :)'

In [45]:
df_train['review'] = df_train['review'].apply(preprocess)

In [46]:
df_train.head(10)

,review,label
0,first low budget movi expect incred low go ass...,0
1,bad b movi masquerad mockumentari porn documen...,0
2,seen better stori predict film crew tri film h...,0
3,channel surf caught logo one watch horribl bad...,0
4,invis ray part scienc fiction part horror also...,1
5,rent movi promis william dafo robert rodriguez...,0
6,direct sequel mummi hand 1940 lead charact ste...,0
7,first collabor schoedsack cooper compel docume...,1
8,day 2002 chase scarecrow felt film handl serio...,1
9,possibl spoiler movi reput preced anticip sat ...,0


In [47]:
df_train.shape

(25000, 2)

In [48]:
#Сохраним тренировочный датаaфрейм в csv файл
df_train.to_csv('../dataset/aclImdb/train/movie_data.csv')


In [3]:
df_train = pd.read_csv('../dataset/aclImdb/train/movie_data.csv', encoding='UTF-8')
df_test = pd.read_csv('../dataset/aclImdb/test/movie_data.csv', encoding='UTF-8')


In [4]:
df_train.drop(['Unnamed: 0'],axis=1, inplace=True)
df_train

,review,label
0,first low budget movi expect incred low go ass...,0
1,bad b movi masquerad mockumentari porn documen...,0
2,seen better stori predict film crew tri film h...,0
3,channel surf caught logo one watch horribl bad...,0
4,invis ray part scienc fiction part horror also...,1
...,...,...
24995,sit movi doubt could make second time mild ent...,0
24996,thought grinch stole christma pretti good movi...,1
24997,film bit tedious most silent film bulk stori p...,0
24998,origin les visiteur origin hilari interest bal...,0


In [5]:
df_test.drop(['Unnamed: 0'],axis=1, inplace=True)
df_test

,review,label
0,begin movi give feel director tri portray some...,0
1,sit watch 14th season bachelor wing love knew ...,0
2,good way baseketbal wast film everi singl way ...,0
3,film travesti wait abomin wait without doubt a...,0
4,warn potenti viewer look adapt classic stori d...,0
...,...,...
24995,carri fisher state one occas made movi period ...,0
24996,guy get whack right plain sight guy shoot got ...,0
24997,guess beat street made nation appear flashdanc...,1
24998,insist martin luther king inspir spirit resid ...,1


In [6]:
# Разобьем тренировочный датасет
X_train, X_test, Y_train, Y_test = train_test_split(df_train['review'],df_train['label'], test_size = 0.25, random_state=0)

In [7]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((18750,), (18750,), (6250,), (6250,))

In [49]:
test_pos_path = '../dataset/aclImdb/test/pos'
test_neg_path = '../dataset/aclImdb/test/neg'
test_pos_path, test_neg_path

('../dataset/aclImdb/test/pos', '../dataset/aclImdb/test/neg')

In [50]:
len(os.listdir(test_pos_path)), len(os.listdir(test_neg_path))

(12500, 12500)

In [51]:
neg_reviews_test = load_reviews_from_files(test_neg_path, 0)
pos_reviews_test = load_reviews_from_files(test_pos_path, 1)
len(neg_reviews_test), len(pos_reviews_test)

(12500, 12500)

In [52]:
test_reviews = neg_reviews + pos_reviews

In [53]:
df_test = pd.DataFrame(test_reviews, columns=['review', 'label'])
df_test

,review,label
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1
...,...,...
24995,"Towards the end of the movie, I felt it was to...",0
24996,This is the kind of movie that my enemies cont...,0
24997,I saw 'Descent' last night at the Stockholm Fi...,0
24998,Some films that you pick up for a pound turn o...,0


In [54]:
df_test = shuffle(df_test)
df_test.reset_index(inplace=True, drop=True)

In [55]:
df_test

,review,label
0,"From the beginning of the movie, it gives the ...",0
1,Sitting down to watch the 14th season of the B...,0
2,...and not in a good way. BASEketball is a was...,0
3,"This ""film"" is a travesty. No, wait--an abomin...",0
4,A warning to potential viewers: if you are loo...,0
...,...,...
24995,Carrie Fisher has stated on more than one occa...,0
24996,Some guy gets whacked. Right out in plain sigh...,0
24997,"I guess when ""Beat Street"" made a national app...",1
24998,Insisting that Martin Luther King's inspiratio...,1


In [57]:
df_test['review'] = df_test['review'].apply(preprocess)

In [58]:
df_test.to_csv('../dataset/aclImdb/test/movie_data.csv')

In [8]:
df_test = pd.read_csv('../dataset/aclImdb/test/movie_data.csv')

In [10]:
X_test_global = df_test['review']
Y_test_global = df_test['label']
X_test_global, Y_test_global

(0        begin movi give feel director tri portray some...
 1        sit watch 14th season bachelor wing love knew ...
 2        good way baseketbal wast film everi singl way ...
 3        film travesti wait abomin wait without doubt a...
 4        warn potenti viewer look adapt classic stori d...
                                ...                        
 24995    carri fisher state one occas made movi period ...
 24996    guy get whack right plain sight guy shoot got ...
 24997    guess beat street made nation appear flashdanc...
 24998    insist martin luther king inspir spirit resid ...
 24999    laugh lot incred bad sorri folk definit one wo...
 Name: review, Length: 25000, dtype: object,
 0        0
 1        0
 2        0
 3        0
 4        0
         ..
 24995    0
 24996    0
 24997    1
 24998    1
 24999    0
 Name: label, Length: 25000, dtype: int64)

# Обучение моделей

In [12]:
tfidf = TfidfVectorizer(preprocessor=None, tokenizer=None)

## Логистическая Регрессия

In [135]:
pipeline_lr = Pipeline([
    ('vect', tfidf),
    ('clf', LogisticRegression(solver='liblinear'))
])

In [136]:
param_grid_lr = [{'vect__ngram_range':[(1,1)],
                 'clf__penalty' : ['l1','l2'],
                  'clf__C':[1.0, 10.0, 100.0]
                 }]

In [137]:
gs_lr_tfidf = GridSearchCV(pipeline_lr, 
                           param_grid=param_grid_lr,
                          cv = 5,
                          verbose = 2,
                          n_jobs=-1)

gs_lr_tfidf.fit(X_train, Y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)]}],
             verbose=2)

In [138]:
gs_lr_tfidf.best_params_

{'clf__C': 1.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1)}

In [139]:
lr = gs_lr_tfidf.best_estimator_

In [140]:
y_pred_train = lr.predict(X_test)

In [141]:
print(classification_report(y_pred_train, Y_test))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89      3099
           1       0.90      0.88      0.89      3151

    accuracy                           0.89      6250
   macro avg       0.89      0.89      0.89      6250
weighted avg       0.89      0.89      0.89      6250



Получили на тренировчном наборе точность почти 90%. Неплохо

In [128]:
y_pred_test_lr = lr.predict(X_test_global)

In [129]:
print(classification_report(y_pred_test_lr, Y_test_global))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92     12258
           1       0.93      0.91      0.92     12742

    accuracy                           0.92     25000
   macro avg       0.92      0.92      0.92     25000
weighted avg       0.92      0.92      0.92     25000



На тестовом множестве точность 92. Отлично!

In [130]:
# Сохраним модель
dump(lr,'../Models/LogisticRegression.joblib')

['../Models/LogisticRegression.joblib']

## Случайный лес

In [142]:
# pipeline_rf = Pipeline([
#     ('vect', tfidf),
#     ('clf', RandomForestClassifier())
# ])

# param_grid_rf = [{
#     'vect__ngram_range':[(1,1)],
#     'clf__n_estimators': [50, 100, 150, 200],
#     'clf__max_depth': [None, 10, 20, 30],
#     'clf__min_samples_split': [2, 5, 10],
#     'clf__min_samples_leaf': [1, 2, 4],
#     'clf__max_features': ['sqrt', 'log2']
# }]

# gs_rf_tfidf = GridSearchCV(pipeline_rf, 
#                            param_grid=param_grid_rf,
#                           cv = 5,
#                           verbose = 2,
#                           n_jobs=-1)

In [143]:
# gs_rf_tfidf.fit(X_train, Y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid=[{'clf__max_depth': [None, 10, 20, 30],
                          'clf__max_features': ['sqrt', 'log2'],
                          'clf__min_samples_leaf': [1, 2, 4],
                          'clf__min_samples_split': [2, 5, 10],
                          'clf__n_estimators': [50, 100, 150, 200],
                          'vect__ngram_range': [(1, 1)]}],
             verbose=2)

In [144]:
# gs_rf_tfidf.best_params_

{'clf__max_depth': None,
 'clf__max_features': 'sqrt',
 'clf__min_samples_leaf': 2,
 'clf__min_samples_split': 10,
 'clf__n_estimators': 200,
 'vect__ngram_range': (1, 1)}

In [159]:
# gs_rf_tfidf.best_score_

0.8512000000000001

In [160]:
# rf = gs_rf_tfidf.best_estimator_
# rf

Pipeline(steps=[('vect', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(min_samples_leaf=2,
                                        min_samples_split=10,
                                        n_estimators=200))])

In [13]:
rf =  RandomForestClassifier( n_estimators=200, max_features='sqrt', min_samples_leaf=2,min_samples_split=10)

In [14]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [16]:
rf.fit(X_train_tfidf, Y_train)

RandomForestClassifier(min_samples_leaf=2, min_samples_split=10,
                       n_estimators=200)

In [17]:
y_pred_rf=rf.predict(X_test_tfidf)
y_pred_rf

array([1, 0, 1, ..., 0, 1, 0], dtype=int64)

In [18]:
print(classification_report(y_pred_rf, Y_test))

              precision    recall  f1-score   support

           0       0.86      0.87      0.86      3116
           1       0.86      0.86      0.86      3134

    accuracy                           0.86      6250
   macro avg       0.86      0.86      0.86      6250
weighted avg       0.86      0.86      0.86      6250



In [19]:
x_test_global_tfidf = tfidf.transform(X_test_global)

In [20]:
y_pred_rf_global=rf.predict(x_test_global_tfidf)

In [21]:
print(classification_report(y_pred_rf_global, Y_test_global))


              precision    recall  f1-score   support

           0       0.96      0.96      0.96     12458
           1       0.96      0.96      0.96     12542

    accuracy                           0.96     25000
   macro avg       0.96      0.96      0.96     25000
weighted avg       0.96      0.96      0.96     25000



In [22]:
rf.predict_proba(x_test_global_tfidf[0]), Y_test_global[0]

(array([[0.79309785, 0.20690215]]), 0)

In [24]:
dump(rf,'../Models/RandomForestClassifier.joblib')

['../Models/RandomForestClassifier.joblib']

In [25]:
dump(tfidf, '../Models/rf_vectorizer.joblib')

['../Models/rf_vectorizer.joblib']

Случайный Лес достиг на тестовой выборке точности 96%

Логистическая регрессия достигла точности 92%